## Scraping Reddit and posting to Pandas

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

In [2]:
# define a scraper function
def lovely_soup(url):
    r = requests.get(url, headers = {'User-agent': 'Agent_Smith'})
    return BeautifulSoup(r.text, 'lxml')

In [3]:
# apply the function to our reddit source
url = 'https://old.reddit.com/r/AskReddit/'
soup = lovely_soup(url)
# note: you can also try this one
# url = 'https://old.reddit.com/r/Showerthoughts?sort=top&t=week'

In [4]:
# create a list of titles
titles = soup.findAll('p', {'class': 'title'})
titleslist=[]
for title in titles:
    titleslist.append(title.text)

In [5]:
# create a list of dates
dates = soup.findAll('time', {'class':"live-timestamp"})
dateslist=[]
for date in dates:
    output = str(date).split('title="')[1].split('2020')[0]
    dateslist.append(output)

In [6]:
# convert the two lists into a pandas dataframe
df_dict={'date':dateslist, 'post':titleslist}
working_df = pd.DataFrame(df_dict)
pd.set_option('display.max_colwidth', 200)
working_df['date'] = working_df['date'].str.strip()

In [7]:
# write a function to clean up the date
def parse_that_date(row):
    x = row.split(' ')[1:]
    y = ' '.join(x)
    z = '2020 '+ y
    return z[:20]
parse_that_date(working_df.iloc[0,0])

'2020 Dec 24 02:58:06'

In [8]:
# apply the date parsing function and sort the dataframe
working_df['cleandate']=working_df['date'].apply(parse_that_date)
working_df['UTC_date'] = pd.to_datetime(working_df['cleandate'])
working_df.sort_values('UTC_date', inplace=True, ascending=False)
final_df = working_df.drop(['date', 'cleandate'], axis=1)[['UTC_date', 'post']]
final_df.head()

,UTC_date,post
13,2020-12-24 11:37:19,Which videogame consumed your entire life upon first play-through? (self.AskReddit)
17,2020-12-24 11:25:52,What do people need to understand about mental illness? (self.AskReddit)
19,2020-12-24 10:49:10,What are some small things your partner does that reminds you that they are still in love with you? (self.AskReddit)
20,2020-12-24 10:28:44,"If someone came to your place and stole what’s on top of the fridge, what would they walk away with? (self.AskReddit)"
18,2020-12-24 10:04:10,"What is the most incredibly powerful/emotional without words song, you have ever heard? (self.AskReddit)"


In [9]:
# write pandas df as html file 
html = final_df.to_html(index=False) 
text_file = open("redditposts.html", "w") 
text_file.write(html) 
text_file.close() 

In [10]:
# inspiration for this notebook:
# https://recycledrobot.co.uk/words/?web-scraping